In [16]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Ruta donde están almacenadas las imágenes
dataset_dir = r'C:\Users\Carol\Documents\GitHub\ONLINE_DS_THEBRIDGE_Cmonzon\05_Deep_Learning\Sprint_17\Team_Challenge\data'

# Preprocesamiento de las imágenes (normalización de píxeles entre 0 y 1)
image_generator = ImageDataGenerator(rescale=1./255)

# Cargar las imágenes desde las carpetas (cada carpeta es una clase)
image_data = image_generator.flow_from_directory(
    dataset_dir,
    target_size=(64, 64),  # Redimensionar todas las imágenes a 64x64 píxeles
    batch_size=32,
    class_mode='categorical',  # Etiquetas categóricas (clasificación múltiple)
    color_mode='rgb',          # Usamos imágenes a color (RGB)
    shuffle=True
)

# Ver las clases detectadas
print(f"Clases en el dataset: {image_data.class_indices}")

# Inicializamos listas para almacenar las imágenes y etiquetas
X_all = []
y_all = []

# Iteramos sobre todos los lotes del generador
for X_batch, y_batch in image_data:
    X_all.append(X_batch)
    y_all.append(y_batch)
    
    # Si hemos procesado todas las muestras, rompemos el bucle
    if len(X_all) * image_data.batch_size >= image_data.samples:
        break

# Convertimos las listas a arrays de numpy
X_all = np.concatenate(X_all, axis=0)
y_all = np.concatenate(y_all, axis=0)

# Dividir el dataset en entrenamiento y test (80% entrenamiento, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)

# Preprocesamiento de datos para clasificador
# Convertir las imágenes a una forma que Scikit-learn pueda usar (por ejemplo, 64x64x3 a 12288)
X_train_flattened = X_train.reshape(X_train.shape[0], -1)  # Redimensionamos las imágenes a vectores 1D
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Crear el Pipeline para el clasificador RandomForest
pipeline_supervisado = Pipeline([
    ('scaler', StandardScaler()),  # Normalización de características
    ('classifier', RandomForestClassifier())  # Clasificador Random Forest
])

# Realizar validación cruzada (usamos cross_val_score)
cross_val_scores = cross_val_score(pipeline_supervisado, X_train_flattened, np.argmax(y_train, axis=1), cv=5)
print(f"Scores de validación cruzada: {cross_val_scores}")
print(f"Precisión media en validación cruzada: {cross_val_scores.mean()}")

# Definir parámetros para optimización de hiperparámetros con GridSearchCV
param_grid = {
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

# Crear el GridSearchCV
grid_search = GridSearchCV(pipeline_supervisado, param_grid, cv=5, n_jobs=-1)

# Entrenar el modelo con los mejores parámetros encontrados por GridSearch
grid_search.fit(X_train_flattened, np.argmax(y_train, axis=1))  # Usamos np.argmax para obtener las etiquetas con la mayor probabilidad

# Imprimir los mejores parámetros encontrados por GridSearchCV
print(f"Mejores parámetros de GridSearch: {grid_search.best_params_}")

# Evaluar el modelo con los mejores parámetros en el conjunto de prueba
y_pred = grid_search.best_estimator_.predict(X_test_flattened)
test_accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
print(f"Precisión en el conjunto de prueba: {test_accuracy}")

# Evaluación de desempeño en el conjunto de test
from sklearn.metrics import classification_report, confusion_matrix

print("Reporte de clasificación:")
print(classification_report(np.argmax(y_test, axis=1), y_pred))

print("Matriz de confusión:")
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))



Found 2626 images belonging to 26 classes.
Clases en el dataset: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25}
Scores de validación cruzada: [0.99285714 0.98809524 0.9952381  0.99761905 0.9952381 ]
Precisión media en validación cruzada: 0.9938095238095238
Mejores parámetros de GridSearch: {'classifier__max_depth': 20, 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 200}
Precisión en el conjunto de prueba: 0.9980988593155894
Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98        24
           1       1.00      1.00      1.00        17
           2       0.96      1.00      0.98        25
           3       1.00      1.00      1.00        21
           4       1.00      1.00      1.00  